In [1]:
!pip install scipy --quiet
!pip install tenacity --quiet
!pip install tiktoken==0.3.3 --quiet
!pip install termcolor --quiet
!pip install arxiv --quiet
!pip install pandas --quiet
!pip install PyPDF2 --quiet
!pip install tqdm --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [2]:
!pip install anthropic --quiet
!pip install sentence_transformers --quiet
!pip install arxiv --quiet
!pip install tenacity --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.5/891.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.5 MB/s eta 0:00:00


In [3]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 32.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [4]:
import os
import arxiv
import ast
import concurrent
import json
import os
import pandas as pd
import tiktoken
from csv import writer
from IPython.display import display, Markdown, Latex
from PyPDF2 import PdfReader
from scipy import spatial
from tenacity import retry, wait_random_exponential, stop_after_attempt
from tqdm import tqdm
from termcolor import colored
import anthropic
from sentence_transformers import SentenceTransformer

ANTHROPIC_MODEL = "claude-3-5-sonnet-20240620"
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key:")
client = anthropic.Anthropic()

Enter your Anthropic API key:··········


In [6]:
directory = './data/papers'

# Check if the directory already exists
if not os.path.exists(directory):
    # If the directory doesn't exist, create it and any necessary intermediate directories
    os.makedirs(directory)
    print(f"Directory '{directory}' created successfully.")
else:
    # If the directory already exists, print a message indicating it
    print(f"Directory '{directory}' already exists.")

Directory './data/papers' created successfully.


In [7]:
# Set a directory to store downloaded papers
data_dir = os.path.join(os.curdir, "data", "papers")
paper_dir_filepath = "./data/arxiv_library.csv"

# Generate a blank dataframe where we can store downloaded files
df = pd.DataFrame(list())
df.to_csv(paper_dir_filepath)

In [8]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def embedding_request(text):
    # Generate embeddings using the SentenceTransformer model
    embedding = embedding_model.encode(text)
    return embedding

@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def get_articles(query, library='./data/arxiv_library.csv', top_k=5):
    """Fetch the top_k articles based on a user's query, sorted by relevance.
    The articles and their embeddings are stored in the library file for later retrieval.
    """
    client = arxiv.Client()
    search = arxiv.Search(
        query=query,
        max_results=10,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    result_list = []
    data_dir = "/content/data/papers"  # Specify your path here

    # Ensure the directory exists
    os.makedirs(os.path.dirname(library), exist_ok=True)

    for result in client.results(search):
        result_dict = {
            "title": result.title,
            "summary": result.summary,
            "article_url": [x.href for x in result.links][0],
            "pdf_url": [x.href for x in result.links][1]
        }
        result_list.append(result_dict)

        # Get the embedding for the article title
        embedding = embedding_request(text=result.title)

        # Store references in the library file
        file_reference = [
            result.title,
            result.download_pdf(data_dir),
            embedding.tolist()  # Convert the numpy array to a list for storing in CSV
        ]

        # Write to file
        with open(library, "a") as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(file_reference)

    return result_list

In [9]:
# Test that the search is working
result_output = get_articles("LLM Safety")
result_output[0]

{'title': 'Controllable Text Generation for Large Language Models: A Survey',
 'summary': "In Natural Language Processing (NLP), Large Language Models (LLMs) have\ndemonstrated high text generation quality. However, in real-world applications,\nLLMs must meet increasingly complex requirements. Beyond avoiding misleading or\ninappropriate content, LLMs are also expected to cater to specific user needs,\nsuch as imitating particular writing styles or generating text with poetic\nrichness. These varied demands have driven the development of Controllable Text\nGeneration (CTG) techniques, which ensure that outputs adhere to predefined\ncontrol conditions--such as safety, sentiment, thematic consistency, and\nlinguistic style--while maintaining high standards of helpfulness, fluency, and\ndiversity.\n  This paper systematically reviews the latest advancements in CTG for LLMs,\noffering a comprehensive definition of its core concepts and clarifying the\nrequirements for control conditions an

In [10]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100,
) -> list[str]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = embedding_request(query)

    # Check the structure of the response and extract the embedding accordingly
    if isinstance(query_embedding_response, dict) and 'data' in query_embedding_response:
        query_embedding = query_embedding_response['data'][0]['embedding']
    elif isinstance(query_embedding_response, list):
        query_embedding = query_embedding_response[0]['embedding']
    else:
        query_embedding = query_embedding_response  # Assume it's already the embedding

    strings_and_relatednesses = [
        (row["filepath"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    return [str_rel[0] for str_rel in strings_and_relatednesses[:top_n]]

In [11]:
from anthropic import HUMAN_PROMPT, AI_PROMPT

def read_pdf(filepath):
    """Takes a filepath to a PDF and returns a string of the PDF's contents"""
    # creating a pdf reader object
    reader = PdfReader(filepath)
    pdf_text = ""
    page_number = 0
    for page in reader.pages:
        page_number += 1
        pdf_text += page.extract_text() + f"\nPage Number: {page_number}"
    return pdf_text


# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    """Returns successive n-sized chunks from provided text."""
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.5 * n and 1.5 * n tokens
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j



def extract_chunk(content, template_prompt):
    """This function applies a prompt to some input content. In this case it returns a summarized chunk of text"""
    messages = [
        {
            "role": "user",
            "content": f"{template_prompt}\n\nHere's the content to summarize:\n\n{content}"
        }
    ]
    response = client.messages.create(
        model=ANTHROPIC_MODEL,
        messages=messages,
        max_tokens=1000,
        temperature=0
    )
    return response.content[0].text

def summarize_text(query):
    """This function does the following:
    - Reads in the arxiv_library.csv file in including the embeddings
    - Finds the closest file to the user's query
    - Scrapes the text out of the file and chunks it
    - Summarizes each chunk in parallel
    - Does one final summary and returns this to the user"""

    # A prompt to dictate how the recursive summarizations should approach the input paper
    summary_prompt = """Summarize this text from an academic paper. Extract any key points with reasoning. Keep any mathematical equations and tables(if any)\n\nContent:"""

    # If the library is empty (no searches have been performed yet), we perform one and download the results
    library_df = pd.read_csv(paper_dir_filepath).reset_index()
    if len(library_df) == 0:
        print("No papers searched yet, downloading first.")
        get_articles(query)
        print("Papers downloaded, continuing")
        library_df = pd.read_csv(paper_dir_filepath).reset_index()
    library_df.columns = ["title", "filepath", "embedding"]
    library_df["embedding"] = library_df["embedding"].apply(ast.literal_eval)
    strings = strings_ranked_by_relatedness(query, library_df, top_n=1)
    print("Chunking text from paper")
    pdf_text = read_pdf(strings[0])

    # Initialise tokenizer
    tokenizer = tiktoken.get_encoding("cl100k_base")
    results = ""

    # Chunk up the document into 1500 token chunks
    chunks = create_chunks(pdf_text, 1500, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    print("Summarizing each chunk of text")

    # Parallel process the summaries
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=len(text_chunks)
    ) as executor:
        futures = [
            executor.submit(extract_chunk, chunk, summary_prompt)
            for chunk in text_chunks
        ]
        with tqdm(total=len(text_chunks)) as pbar:
            for _ in concurrent.futures.as_completed(futures):
                pbar.update(1)
        for future in futures:
            data = future.result()
            results += data

    # Final summary
    print("Summarizing into overall summary")
    response = client.messages.create(
        model=ANTHROPIC_MODEL,
        messages=[
            {
                "role": "user",
                "content": f"""Write a summary collated from this collection of key points extracted from an academic paper.
                        The summary should highlight the core argument, conclusions and evidence, and answer the user's query.
                        User query: {query}
                        The summary should be structured in bulleted lists following the headings Core Argument, Evidence, and Conclusions. Keep all the mathematical equations
                        Key points:\n{results}\nSummary:\n"""
            }
        ],
        max_tokens=1000,
        temperature=0
    )
    return response.content[0].text

In [15]:
# Test the summarize_text function works

chat_test_response = summarize_text("LLM Safety")

Chunking text from paper
Summarizing each chunk of text


100%|██████████| 8/8 [00:10<00:00,  1.32s/it]


Summarizing into overall summary


In [16]:
print(chat_test_response)

Core Argument:
• DLCRec is a novel approach for managing diversity in Large Language Model (LLM)-based recommender systems that enables fine-grained control over diversity while maintaining high accuracy.
• It uses a task decomposition strategy, breaking the recommendation process into three sub-tasks (genre prediction, genre filling, item prediction) that are trained independently and inferred sequentially based on user-defined control numbers.

Evidence:
• Comprehensive empirical evaluation on two real-world datasets (MovieLens10M and Steam) demonstrates:
  - DLCRec achieves the lowest MAE_Cov@10 in all scenarios, indicating best adherence to diversity control numbers
  - Minor accuracy trade-offs for better controllability compared to baselines
  - Decomposition framework is crucial for performance - combining tasks led to poorer results
  - Data augmentation strategies improved performance on genre filling and prediction tasks
• Mathematical formulation of the autoregressive loss f

In [73]:
import json
from anthropic import Anthropic

client = Anthropic()

def generate_latex_paper(query, summary):
    """
    Generate a .tex file for an academic paper based on summarized research.

    :param query: String, the original research query
    :param summary: String, the summary of analyzed papers
    :return: String, the content of the .tex file
    """

    paper_structure_prompt = f"""Based on the following summary of research papers about "{query}",
    generate a new academic paper. Include a title, authors (fictional),
    abstract, and sections (including introduction, methodology, results, discussion, and conclusion).
    For each section, provide a solid description of what should be included in a research paper format.
    All the sections must be realistic albeit short and feel like a real scientific paper.
    Do not give guidelines to write the sections but write the section completely.
    It should feel like a real 2-3 page research paper that could be submitted to a conference.
    Incorporate the mathematical equations of the paper in the sections, the way you would write them in a .tex file. Add tables with fictional results if applicable.
    Keep placeholders of two images that could be added later and add descriptions of the images as the figure description. Do the same for the tables.
    The image placeholders should be inside square parentheses and start with the keyword IMAGE PLACEHOLDER followed by the serial number.
    It would also contain a detailed description of the image that needs to be generated. The image could either be a random image with the title [picture] or a chart.
    If the image is a chart. It should contain two informations: 1) data_prompt and 2) plot_prompt. Here are examples of the two.

        [data_prompt] = The data shows the heights in inches of 30 children aged 12 years old.
        The data has 2 columns labeled 'Height' and 'Gender'.
        The values should show realistic heights for American children,
        [plot_prompt] = Histogram with stacked bars. One set of bars is pink for female, and the other set is baby blue for male.
        The legend labels the colors.

        the descriptions must have spaces between them and not be juxtaposed together.

        This is just for reference. The actual description must be in line with the paper that you are generating.

      Initially add a word in parentheses to mention if it's a (picture) or a (chart) before generating the description.

      All of this information would be encapsulated inside square brackets containing the IMAGE PLACEHOLDER, then serial number, then the type: chart or picture, then the description of data_prompt and plot_prompt if it is a chart.

    The table should be in LaTeX format, using the tabular environment.
    The math equations should be written the same way as it is done for LaTeX as well.
    Follow the structure of the paper rigorously, do not miss out on sections.
    Use LaTeX citation commands (\\cite{{}}) throughout the paper where appropriate.
    Include a 'references' key in the JSON with a list of at least 5 relevant references. Each reference should be a string in BibTeX format.
    Ensure all required keys (title, authors, abstract, sections, references) are included in the JSON structure.

    Generate the response as a valid JSON object.  It is crucial that you do not include any control characters (ASCII values 0-31) in the JSON. All strings must be properly escaped.
    For the 'sections' key, use a list of dictionaries, where each dictionary has 'name' and 'content' keys.
    Ensure all string values are properly escaped, especially when including LaTeX commands or special characters.


    Summary of research:
    {summary}

    Respond with a JSON structure containing the complete paper content. Make sure that there are no unterminated strings at the end of the json. I don't want any JSON Decode Error. Having a correct json is the primary objective."""

    structure_response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        messages=[{"role": "user", "content": paper_structure_prompt}],
        max_tokens=4000
    )

    # Extract JSON from the response
    paper_structure_text = structure_response.content[0].text

    try:
        paper_structure = json.loads(paper_structure_text)
    except json.JSONDecodeError as e:
        print("JSON Decode Error:", str(e))
        # Use Claude to fix the broken JSON
        fix_json_prompt = f"""The following JSON structure is invalid. Please fix it and return only the corrected JSON. Ensure all string values are properly escaped, especially LaTeX commands and special characters:

        {paper_structure_text}
        Make sure there are no unterminated strings for the json. The json should contain arguments called 'title', 'authors', 'abstract', 'sections', 'references'. Very slightly reduce length of the paper to create a correct json with the given parameters. Keep tables as mentioned before and also the image placeholders with their descriptions. Remember you are generating everything for a .tex and not a markdown. So follow latex commands as and when necessary."""

        fix_json_response = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            messages=[{"role": "user", "content": fix_json_prompt}],
            max_tokens=4000
        )

        fixed_json_text = fix_json_response.content[0].text

        try:
            paper_structure = json.loads(fixed_json_text)
        except json.JSONDecodeError as e:
            raise ValueError(f"Unable to fix JSON structure: {str(e)}")

    if not isinstance(paper_structure, dict):
        raise TypeError(f"Expected paper_structure to be a dict, but got {type(paper_structure)}")

    required_keys = ['title', 'authors', 'abstract', 'sections', 'references']
    missing_keys = [key for key in required_keys if key not in paper_structure]
    if missing_keys:
        raise ValueError(f"Missing required keys in paper_structure: {', '.join(missing_keys)}")

    # Generate LaTeX content
    latex_content = r"""
\documentclass[12pt,letterpaper]{article}
\usepackage[utf8]{inputenc}
\usepackage{amsmath}
\usepackage{graphicx}
\usepackage[margin=1in]{geometry}
\usepackage{natbib}
\usepackage{booktabs}
\usepackage{float}

\title{%s}
\author{%s}

\begin{document}

\maketitle

\begin{abstract}
%s
\end{abstract}

""" % (paper_structure['title'], r" \and ".join(paper_structure['authors']), paper_structure['abstract'])

    sections = paper_structure['sections']
    if isinstance(sections, list):
        for section in sections:
            if isinstance(section, dict):
                name = section.get('name', 'Untitled Section')
                content = section.get('content', '')
                latex_content += r"\section{%s}" % name
                latex_content += "\n\n"
                latex_content += content
                latex_content += "\n\n"
            else:
                latex_content += r"\section{Untitled Section}"
                latex_content += "\n\n"
                latex_content += str(section)
                latex_content += "\n\n"
    else:
        raise ValueError("Expected 'sections' to be a list of dictionaries")

    # Add bibliography
    latex_content += r"\bibliographystyle{plain}" + "\n"
    latex_content += r"\begin{thebibliography}{99}" + "\n\n"

    for i, ref in enumerate(paper_structure['references'], 1):
        # Extract the citation key from the BibTeX entry
        match = re.search(r'@\w+{([^,]+),', ref)
        if match:
            citation_key = match.group(1)
        else:
            citation_key = f"ref{i}"

        # Add the bibitem with the correct citation key
        latex_content += f"\\bibitem{{{citation_key}}}\n"

        # Remove the entry type and citation key from the start of the entry
        cleaned_ref = re.sub(r'@\w+{[^,]+,\s*', '', ref)
        # Remove the closing brace of the BibTeX entry
        cleaned_ref = re.sub(r'\}\s*$', '', cleaned_ref)

        latex_content += cleaned_ref + "\n\n"

    latex_content += r"\end{thebibliography}" + "\n\n"
    latex_content += r"\end{document}"

    return latex_content

In [41]:
# Example usage
query = "LLM Safety"
summary = summarize_text(query)  # Assuming this function exists and returns a summary



Chunking text from paper
Summarizing each chunk of text


  0%|          | 0/8 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [74]:
latex_content = generate_latex_paper(query, summary)

# Write the content to a .tex file
with open("generated_paper_with_ref7.tex", "w") as f:
    f.write(latex_content)

print("LaTeX file 'generated_paper_with_ref7.tex' has been generated.")

LaTeX file 'generated_paper_with_ref7.tex' has been generated.


In [75]:
# prompt: read and print the .tex file

with open("generated_paper_with_ref7.tex", "r") as f:
    latex_content = f.read()

print(latex_content)



\documentclass[12pt,letterpaper]{article}
\usepackage[utf8]{inputenc}
\usepackage{amsmath}
\usepackage{graphicx}
\usepackage[margin=1in]{geometry}
\usepackage{natbib}
\usepackage{booktabs}
\usepackage{float}

\title{DLCRec: A Task-Decomposed Approach for Controllable Diversity in LLM-Based Recommender Systems}
\author{Sophia Zhang \and Alexander Chen \and Olivia Patel \and David Lee}

\begin{document}

\maketitle

\begin{abstract}
Large Language Models (LLMs) have shown remarkable potential in recommendation systems, but managing diversity while maintaining accuracy remains a challenge. This paper introduces DLCRec, a novel approach that enables fine-grained control over diversity in LLM-based recommender systems. By decomposing the recommendation task into genre prediction, genre filling, and item prediction sub-tasks, and employing data augmentation techniques, DLCRec achieves precise control over diversity with minimal impact on accuracy. We evaluate our approach on two real-world 